In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import re

In [74]:
mass = pd.read_csv('../source/mmALL_073120_csv.csv')
mass.head(3)

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,protesterdemand4,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes
0,201990001,Canada,20,1990,North America,1,1,15.0,1.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...
1,201990002,Canada,20,1990,North America,1,2,25.0,6.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...
2,201990003,Canada,20,1990,North America,1,3,1.0,7.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai..."


In [75]:
mass = mass[mass['protest']==1]

In [76]:
mass.participants_category.value_counts()

100-999       3204
50-99         2508
2000-4999     1580
>10000        1470
5000-10000     642
1000-1999      483
Name: participants_category, dtype: int64

In [77]:
mass.participants_category.isnull().sum()

5352

In [78]:
mass['participants'].isnull().sum()

12

In [79]:
def string_to_number(string):

    # if nan then just exit
    if string is np.nan:
        return string
    
    output = re.findall (  r'\d+(?:,\d+)?' , string)
        
    if len(output ) == 0:
        return string
    elif len(output) == 1:
        return output[0]
    elif len(output) > 1:
        return max(output)
    else:
        return output

In [80]:
'tens of thousands'.split(' ')

['tens', 'of', 'thousands']

In [88]:

#Used to convert participants column from a mess of text into nice clean numbers
# Return value is still a string value, but can be converted easily later

def participants_to_numbers(string):

    # if nan then just exit
    if string is np.nan:
        return string
    
    if type(string) is list:  # in case list of numbers
        return string
    
    string = string.lower()
    
    small_groups = ['dozen','dozens', 'a group', 'group','several dozen','scores','people','crowd','crowds']   # 50 -99
    medium_groups = ['busloads','hundreds','widespread','several hundred','large','hundred','protests','demonstrators','teachers','.']  #100 - 999
    large_groups = ['thousand','thousands','several thousand', 'mass', 'huge'] #1000-5000
    very_large_groups = ['tens of thousands']  #5000-10_000
    huge_groups =  ['government']        #10_000 - 100_000
    massive_groups = ['hundreds of thousands','millions','million','venezuela']    # >  100_000
    
    # try to match the incoming string as itself first
    if string in small_groups:
        return 50
    elif string in medium_groups:
        return 200
    elif string in large_groups:
        return 1500
    elif string in very_large_groups:
        return 20_000
    elif string in huge_groups:
        return 50_000
    elif string in massive_groups:
        return 600_000
    
    # if the string is not found in above word groups, split it up as a list of words
    word_list = string.split(' ')
    
    word_list.reverse()   # to handle cases like hundreds of thousands that might fall through
    
    for word in word_list:
        
        word = word.replace('\"','')
        
        if word in small_groups:
            return 50
        elif word in medium_groups:
            return 200
        elif word in large_groups:
            return 1500
        elif word in very_large_groups:
            return 20_000
        elif word in huge_groups:
            return 50_000
        elif word in massive_groups:
            return 600_000
    
    return string

In [82]:
def participants_to_categories(participants):

    # In this function participants should be an int
   
    # if 0 then just exit - but this should probably never happen
    if participants == 0:
        return participants
    
    if participants > 100_000:
        return 'Over 100,000'
    elif participants >= 10_000:
        return '10,000-100,000' 
    elif participants >= 5_000:
        return '5,000-9,999'
    elif participants >= 1000:
        return '1,000-4,999'
    elif participants >= 100:
        return '100-999'
    elif participants >= 50:
        return '50-99'
    else:
        return 'Less than 50'


In [83]:

#mass['participants_converted'] = mass['participants']
mass['participants_converted'] = mass['participants'].apply(string_to_number)
mass['participants_converted'] = mass['participants_converted'].apply(participants_to_numbers)

In [84]:
mass = mass[mass['participants'].notnull()]

mass['participants_converted'] = mass['participants_converted'].replace(',','', regex=True)

mass['participants_converted']= mass['participants_converted'].astype('int64') 

mass['participants_cat_converted'] = mass['participants_converted'].apply(participants_to_categories)


In [87]:
#mass[mass['participants_converted'] > 0]   [['country','year','participants_category','participants','participants_converted','participants_cat_converted','protesterdemand1','stateresponse1']].iloc[50:100]
#mass[mass['participants_category'] != mass['participants_cat_converted']]   [['country','year','participants_category','participants','participants_converted','participants_cat_converted','protesterdemand1','stateresponse1']].iloc[50:100]

In [89]:
mass[ (mass['participants_category'] != mass['participants_cat_converted']) & mass['participants_category'].notna()] [['country','year','participants_category','participants','participants_converted','participants_cat_converted','protesterdemand1','stateresponse1']].iloc[150:200]

,country,year,participants_category,participants,participants_converted,participants_cat_converted,protesterdemand1,stateresponse1
1228,Colombia,2019,>10000,tens of thousands,20000,"10,000-100,000","political behavior, process",crowd dispersal
1229,Colombia,2019,>10000,tens of thousands,20000,"10,000-100,000","political behavior, process",crowd dispersal
1479,Venezuela,2014,>10000,10000,10000,"10,000-100,000","political behavior, process",crowd dispersal
1484,Venezuela,2014,5000-10000,5000,5000,"5,000-9,999","political behavior, process",crowd dispersal
1488,Venezuela,2014,>10000,10000s,10000,"10,000-100,000","political behavior, process",crowd dispersal
1490,Venezuela,2014,5000-10000,5000,5000,"5,000-9,999","price increases, tax policy",crowd dispersal
1494,Venezuela,2015,2000-4999,1000s,1000,"1,000-4,999","political behavior, process",ignore
1502,Venezuela,2016,>10000,Thousands,1500,"1,000-4,999",removal of politician,crowd dispersal
1503,Venezuela,2016,100-999,several dozen arrests,50,50-99,"price increases, tax policy",arrests
1504,Venezuela,2016,5000-10000,several thousand,1500,"1,000-4,999",removal of politician,crowd dispersal


In [90]:
mass['participants_cat_converted'].value_counts()

100-999           4746
1,000-4,999       3897
50-99             3493
10,000-100,000    2121
5,000-9,999        576
Over 100,000       369
Less than 50        25
Name: participants_cat_converted, dtype: int64